# Sentiment Classification of Movie Reviews

This notebook builds a sentiment classifier to predict positive or negative sentiment from Rotten Tomatoes movie reviews using the Sentence Polarity dataset. The workflow covers data loading, text preprocessing, TF-IDF feature extraction, model training with hyperparameter tuning, and evaluation with K-fold cross-validation.

## 1 - Reading in & Preprocessing the Data

### 1.1 Importing Libraries

In [7]:
# Importing necessary libraries
import numpy as np  # for numerical operations
import pandas as pd  # for data manipulation
import random  # for shuffling the data
import nltk
import re  # for handling regular expressions

from nltk.stem import WordNetLemmatizer  # for lemmatizing words
from nltk.corpus import stopwords  # for stop word removal
from nltk.tokenize import word_tokenize  # for tokenizing sentences into words
nltk.download('punkt_tab')  # Downloads the 'punkt' tokenizer table used for tokenization of text into sentences or words

# Downloading necessary NLTK resources
nltk.download('stopwords')  # List of common stop words in English
nltk.download('punkt')  # Pre-trained tokenizer models
nltk.download('wordnet')  # WordNet lemmatizer dataset

# Libraries for text feature extraction and model training
from sklearn.feature_extraction.text import TfidfVectorizer  # Convert text into numerical features (TF-IDF)
from sklearn.linear_model import LogisticRegression  # Logistic regression for classification
from sklearn.svm import LinearSVC  # Support Vector Machines for classification

# Libraries for model evaluation
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix  # For model evaluation metrics
from sklearn.model_selection import KFold, cross_val_score  # For cross-validation

[nltk_data] Downloading package punkt_tab to /Users/annie/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to /Users/annie/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/annie/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/annie/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


### 1.2 Read Data Files

The `Sentence Polarity` dataset contains 5,331 positive and 5,331 negative sentences. We'll load this dataset and prepare it for analysis.

In [8]:
# Read the positive and negative sentiment files
df_sent_pos = pd.read_csv('https://raw.githubusercontent.com/chrisvdweth/nus-cs4248x/refs/heads/master/1-foundations/data/corpora/sentence-polarity-dataset/sentence-polarity.neg', sep='\t', header=None)  # Positive sentiment sentences
df_sent_neg = pd.read_csv('https://raw.githubusercontent.com/chrisvdweth/nus-cs4248x/refs/heads/master/1-foundations/data/corpora/sentence-polarity-dataset/sentence-polarity.pos', sep='\t', header=None)  # Negative sentiment sentences

# Display the first few rows of the positive dataset to understand its structure
print(df_sent_pos.head())

                                                   0
0                  simplistic , silly and tedious . 
1  it's so laddish and juvenile , only teenage bo...
2  exploitative and largely devoid of the depth o...
3  [garbus] discards the potential for pathologic...
4  a visually flashy but narratively opaque and e...


### 1.3 Rename Columns

In [9]:
# Rename the column to 'sentence' for clarity
df_sent_pos.rename(columns={0: "sentence"}, inplace=True)
df_sent_neg.rename(columns={0: "sentence"}, inplace=True)

### 1.4 Data Preprocessing

The sentences is preprocessed by defining a function called `preprocess_text` that performs the following:

1. Converts text to lowercase.
2. Removes punctuation using regular expressions.
3. Removes extra whitespace.
4. Tokenizes sentences into words.
5. Removes stop words.
6. Lemmatizes words.

In [10]:
# Define the preprocessing function
def preprocess_text(sentences):
    # Convert all tokens to lowercase
    sentences = [sentence.lower() for sentence in sentences]

    # Remove punctuation using regex
    sentences = [re.sub(r"[^\w\s]", "", sentence) for sentence in sentences]

    # Remove extra whitespace between words
    sentences = [" ".join(sentence.split()) for sentence in sentences]

    # Tokenize sentences into words
    sentences = [word_tokenize(sentence) for sentence in sentences]

    # Remove stop words
    stop_words = set(stopwords.words('english'))  # Load English stop words
    filtered_sentences = []
    for sentence in sentences:
        filtered_sentence = [word for word in sentence if word not in stop_words]
        filtered_sentences.append(filtered_sentence)

    # Lemmatize words
    lemmatizer = WordNetLemmatizer()
    lemmatized_sentences = []
    for sentence in filtered_sentences:
        lemmatized_sentence = [lemmatizer.lemmatize(word) for word in sentence]
        lemmatized_sentences.append(lemmatized_sentence)

    return [' '.join(sentence) for sentence in lemmatized_sentences]

### 1.5 Apply Preprocessing

Preprocessing function is applied to both negative and positive sentences.

In [11]:
# Preprocess the sentences
pos_preprocessed_sentences = preprocess_text(df_sent_pos['sentence'])
neg_preprocessed_sentences = preprocess_text(df_sent_neg['sentence'])

# Print the first preprocessed negative sentence
print(neg_preprocessed_sentences[0])

rock destined 21st century new conan he going make splash even greater arnold schwarzenegger jeanclaud van damme steven segal


### 1.6 Combine Dataset

The negative and positive sentences are merged into a single list called `sentences`.

In [12]:
# Combine preprocessed positive and negative sentences
sentences = pos_preprocessed_sentences + neg_preprocessed_sentences

### 1.7 Create Labels

Labels (also targets) distinguish negative (labeled as `0`) and positive (labeled as `1`) sentences.

In [13]:
# Create a list for all labels
polarities = []
polarities.extend([0] * len(df_sent_neg))  # Label negative sentences as 0
polarities.extend([1] * len(df_sent_pos))  # Label positive sentences as 1

### 1.8 Shuffle Data

Randomly shuffle the dataset to ensure sentence are in random order.

In [14]:
# Combine sentences and labels into a single list
combined = list(zip(sentences, polarities))

# Shuffle the combined list
random.shuffle(combined)

# Split the shuffled list back into sentences and labels
sentences[:], polarities[:] = zip(*combined)

### 1.9 Split Dataset

The dataset is split into 80% for training and 20% for testing.

In [15]:
# Define train-test split ratio
train_test_ratio = 0.8

# Calculate the size of the training set
train_set_size = int(train_test_ratio * len(sentences))

# Split data into training and test sets
X_train, X_test = sentences[:train_set_size], sentences[train_set_size:]
y_train, y_test = polarities[:train_set_size], polarities[train_set_size:]

# Print sizes of training and test sets
print("Size of training set:", len(X_train))
print("Size of test set:", len(X_test))

Size of training set: 8529
Size of test set: 2133


## 2 - Vectorizing Texts, Training Models & Evaluating Their Performance

### 2.1 Transforming Text into Features

To use text data in machine learning models, we need to convert it into a numerical format that algorithms can process. The TF-IDF (Term Frequency-Inverse Document Frequency) Vectorizer transforms our preprocessed text into numerical features by measuring how important each term is within a sentence while reducing the weight of commonly occurring words.

In [16]:
# Import TF-IDF Vectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

# Define the vectorizer with default parameters
tfidf_vectorizer = TfidfVectorizer()

# Transform the training data into a TF-IDF matrix
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)

# Check the number of samples and features
num_samples, num_features = X_train_tfidf.shape
print("#Samples: {}, #Features: {}".format(num_samples, num_features))

#Samples: 8529, #Features: 16471


In [20]:
# Diagnostic check - compare vocabulary size
from collections import Counter
all_words = ' '.join(X_train).split()
unique_words = set(all_words)
print(f"Unique words in training set: {len(unique_words)}")
print(f"TF-IDF features: {X_train_tfidf.shape[1]}")
print(f"Number of stopwords: {len(stopwords.words('english'))}")

Unique words in training set: 16501
TF-IDF features: 16471
Number of stopwords: 198


### 2.2 Train the Classifier

Logistic Regression is a simple yet effective algorithm for binary classification tasks, such as predicting sentiment polarity.

In [18]:
# Import the Logistic Regression model
from sklearn.linear_model import LogisticRegression

# Train the Logistic Regression classifier
logistic_regression_classifier = LogisticRegression().fit(X_train_tfidf, y_train)

### 2.3 Evaluation the Classifier

After training the model, we need to assess its performance on unseen test data. Evaluation involves transforming the test data into the same TF-IDF format as the training data, making predictions, and calculating key metrics.

In [19]:
# Transform the test data into TF-IDF format
X_test_tfidf = tfidf_vectorizer.transform(X_test)

# Predict polarities for the test data
y_pred = logistic_regression_classifier.predict(X_test_tfidf)

# Import evaluation metrics
from sklearn.metrics import classification_report, accuracy_score

# Generate and display the classification report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.76      0.75      0.75      1060
           1       0.76      0.76      0.76      1073

    accuracy                           0.76      2133
   macro avg       0.76      0.76      0.76      2133
weighted avg       0.76      0.76      0.76      2133



The Logistic Regression model achieved an F1-score of 0.76 for sentiment classification, demonstrating significantly better performance than random guessing (~50%) with 76% precision and recall for both classes. While this result shows the model performs well above baseline, it indicates there is still room for improvement. The next steps involve exploring cross-validation and hyperparameter tuning techniques to further enhance the model's performance.